# Draft

In [75]:
import pandas as pd
import numpy as np

pd.options.display.max_rows = 1_000

### Pool Settings

In [76]:
# Bitcoin/Thugz League
TEAMS = 12
ROSTER = {'C': 2, 'LW': 2, 'RW': 2, 'D': 4, 'G': 2}

SKATER_CATEGORIES = {
    'goals': 1,     
    'assists': 1, 
    'powerplay_points': 1, 
    'shots_on_goal': 1, 
    'hits': 1,
    'blocks': 1,
    'game_winning_goals': 1/2
}

GOALIE_CATEGORIES = {
    "wins" : 1,            # good team is good
    "save_percentage": 1, 
    "saves": 1, 
    "shutouts": 1/10
}

### Players

In [77]:
pic = pd.read_csv("../data/adp-yahoo_com.csv")
cap = pd.read_csv("../data/info-capfriendly_com.csv")
players = pd.merge(pic, cap, how="left", on="name")

players["adp"] = players["pick"] 
players = players[['team', 'age', 'name', 'position', 'adp']]

players.head()

,team,age,name,position,adp
0,Edm,24.0,Connor McDavid,C,1.1
1,Edm,25.0,Leon Draisaitl,"C,LW",2.4
2,Col,26.0,Nathan MacKinnon,C,3.1
3,TB,28.0,Nikita Kucherov,RW,4.4
4,TB,27.0,Andrei Vasilevskiy,G,5.1


### Multiple Positions

In [78]:
multi = (
    players
    [["name", "position"]]
    .set_index(['name']) 
    .apply(lambda col: col.str.split(',').explode())
    .reset_index()
)

multi.head(20)

,name,position
0,Connor McDavid,C
1,Leon Draisaitl,C
2,Leon Draisaitl,LW
3,Nathan MacKinnon,C
4,Nikita Kucherov,RW
5,Andrei Vasilevskiy,G
6,Mikko Rantanen,RW
7,David Pastrnak,RW
8,Auston Matthews,C
9,Artemi Panarin,LW


### Projection Data

In [79]:
proj = pd.read_csv("../data/projections-generated.csv")

### Goalies

In [80]:
goalies = proj[proj["position"] == "G"].copy()

goalies = (
    goalies
    [["name"] + list(GOALIE_CATEGORIES.keys())]
    .set_index("name")
    .apply(lambda x: (x - x.min()) / (x.max() - x.min()))
)

for key, value in GOALIE_CATEGORIES.items():
    goalies[key] *= value
    
goalies["rollup"] = goalies.apply(lambda row: row.sum(), axis=1)
goalies["rollup"] /= sum(GOALIE_CATEGORIES.values())
goalies["rollup"] *= 100

goalies = goalies.reset_index()
goalies = goalies[["name", "rollup"]]

goalies.head()

,name,rollup
0,Andrei Vasilevskiy,75.749267
1,Robin Lehner,83.225895
2,Darcy Kuemper,81.812159
3,Connor Hellebuyck,81.395085
4,Igor Shesterkin,74.970728


### Skaters

In [81]:
skaters = proj[proj["position"] != "G"].copy()
skaters = (
    skaters
    [["name"] + list(SKATER_CATEGORIES.keys())]
    .set_index("name")
    .apply(lambda x: (x - x.min()) / (x.max() - x.min()))
)
     
for key, value in SKATER_CATEGORIES.items():
    skaters[key] *= value
    
skaters["rollup"] = skaters.apply(lambda row: row.sum(), axis=1)
skaters["rollup"] /= sum(SKATER_CATEGORIES.values())
skaters["rollup"] *= 100

skaters = skaters.reset_index()
skaters = skaters[["name", "rollup"]]

skaters.head()

,name,rollup
0,Connor McDavid,64.562208
1,Leon Draisaitl,57.921437
2,Nathan MacKinnon,59.566783
3,Nikita Kucherov,52.101013
4,Mikko Rantanen,47.982502


### Merge

In [82]:
rollup = pd.concat([skaters, goalies])
rollup = pd.merge(multi, rollup, how="inner", on="name").sort_values("rollup", ascending=False)

### VORP

In [83]:
vorp = rollup.copy()
vorp["vorp"] = vorp["rollup"]

for position, slots in ROSTER.items():
    replacement = (
        vorp[vorp['position'] == position]
        .sort_values('vorp', ascending=False)
        .head(slots * TEAMS)
        ['vorp']
        .mean()
    )
    vorp.loc[vorp['position'] == position, 'vorp'] = vorp['vorp'] - replacement

vorp.head(10)

,name,position,rollup,vorp
19,Robin Lehner,G,83.225895,13.802929
21,Darcy Kuemper,G,81.812159,12.389192
23,Connor Hellebuyck,G,81.395085,11.972118
79,Philipp Grubauer,G,78.849332,9.426365
67,Juuse Saros,G,77.356207,7.933241
5,Andrei Vasilevskiy,G,75.749267,6.326300
45,Igor Shesterkin,G,74.970728,5.547762
49,Marc-Andre Fleury,G,74.654355,5.231389
54,Semyon Varlamov,G,71.549123,2.126156
77,Cam Talbot,G,70.622517,1.199551


In [84]:
lineups = pd.read_csv("../data/lineups-dailyfaceoff_com.csv")
lineups["name"] = lineups["name"].replace("Nathan Mackinnon", "Nathan MacKinnon")

### Draft

In [85]:
draft = pd.merge(players, vorp, how="left", on="name", suffixes=("_yahoo", ""))
draft = pd.merge(draft, lineups, how="left", on="name")
draft = draft.groupby("name").head(1)

In [86]:
draft = draft[[
    'team', 'age', 'name', 
    'line', "ppu",
    'position_yahoo', 'position',
    'rollup', 'vorp', 'adp',
]]

draft["rollup"] = draft["rollup"].round(1)
draft["vorp"] = draft["vorp"].round(1)

draft['pr_vorp'] = draft.groupby("position")['vorp'].rank(ascending=False)
draft['pr_adp'] = draft.groupby("position")['adp'].rank(ascending=True)

draft = draft.sort_values("adp", ascending=True)

In [87]:
draft["pr_arbitrage"] =  draft['pr_adp'] - draft['pr_vorp']
draft['round'] = draft["adp"] // TEAMS + 1

In [88]:
draft.sort_values("pr_vorp", ascending=True)

,team,age,name,line,ppu,position_yahoo,position,rollup,vorp,adp,pr_vorp,pr_adp,pr_arbitrage,round
0,Edm,24.0,Connor McDavid,1.0,1.0,C,C,64.6,21.6,1.1,1.0,1.0,0.0,1.0
25,Was,31.0,John Carlson,1.0,1.0,D,D,43.9,11.8,24.7,1.0,2.0,1.0,3.0
7,Bos,25.0,David Pastrnak,1.0,1.0,RW,RW,53.5,13.4,7.8,1.0,3.0,2.0,1.0
19,VGK,30.0,Robin Lehner,NaN,NaN,G,G,83.2,13.8,18.8,1.0,2.0,1.0,2.0
11,Was,36.0,Alex Ovechkin,1.0,1.0,LW,LW,55.5,14.1,11.3,1.0,3.0,2.0,1.0
3,Col,26.0,Nathan MacKinnon,1.0,1.0,C,C,59.6,16.6,3.1,2.0,3.0,1.0,1.0
4,TB,28.0,Nikita Kucherov,1.0,1.0,RW,RW,52.1,12.0,4.4,2.0,1.0,-1.0,1.0
21,Col,31.0,Darcy Kuemper,NaN,NaN,G,G,81.8,12.4,21.8,2.0,3.0,1.0,2.0
32,TB,30.0,Victor Hedman,1.0,1.0,D,D,43.1,11.1,28.7,2.0,4.0,2.0,3.0
10,Bos,33.0,Brad Marchand,1.0,1.0,LW,LW,47.5,6.1,10.3,2.0,2.0,0.0,1.0


In [89]:
draft = pd.merge(draft, proj.drop(["team", "position"], axis=1))

### Export

In [90]:
draft.to_csv("../drafts/draft-thugz_league.csv", index=False)